In [ ]:
from numpy import *
import time
import sys
import scipy.io

print("enter file name: ", end="")
fil = input()  

fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does not exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

c=mat['c']
A=mat['A']
b=mat['b']
bix=asarray(mat['bix'],dtype=int)
zcheat=mat['zcheat']
if 'xcheat' in mat:
    xcheat=mat['xcheat']
else:
    xcheat=0

bix=bix-1
[m,n] = shape(A)
print('Rows: '+repr(m)+' cols: '+repr(n))

print('c:',c)
print('b:',b)
print('A:',A)
print('bix:',bix)
print('xcheat:',xcheat)
print('zcheat:',zcheat)



In [115]:
from ctypes import sizeof
import numpy as np
import time
import sys

rc_min = np.nanmin(c)


t1=time.time()

[m,n] = np.shape(A)
print( 'Rows: '+repr(m)+' cols: '+repr(n))

# Create nix
nix=np.setdiff1d(range(n), bix) 

B  = A[:, bix]
N  = A[:, nix]
cB = c[bix]
cN = c[nix]
# Start iterations
iter = 0
while iter >= 0:
    iter+=1
    inverseB = np.linalg.inv(B)
    bHatt = np.matmul(inverseB, b)
    print("bhatt origin: ", bHatt)
    print("bix origin: ", bix)
    y = np.transpose(np.matmul(np.transpose(cB), inverseB))

    z= np.matmul(np.transpose(b),y)
    cNhatt = (cN - np.matmul(np.transpose(N),y))

 
    inkvar = int(np.where(cNhatt==np.nanmin(cNhatt))[0][0]) #Tidigare cKhatt min
    rc_min = cNhatt[inkvar]

  
    # calc right-hand-sides and reduced costs
    
     
    # --------

    # calc most negative reduced cost, rc_min,
    # and index for entering variable, inkvar
    # --------


    if rc_min >= -1.0E-12:
        print ('Ready')
        iter=-1
 
        # construct solution, x, and check it
        # --------
       
        x = np.full([np.size(c)], 0.00)
        print("bhatt: ", bHatt)
        

        p = bix.argsort()
        # x =  np.add(x, )
        t = bHatt[p]
        x[:len(t)] += t

        print("t: ", x)
        
        z = np.matmul(np.transpose(b),y)
        print("funktionsvärde: ",z)

        diffx = np.linalg.norm(x-xcheat)
        diffz = z-zcheat
        print ('xdiff: '+repr(diffx))
        print ('zdiff: '+repr(diffz))
    else:
        # calc entering column, a
        # -------- 
        ak = np.matmul(inverseB, N[:, inkvar])
        print("ak: ",ak)
        if max(ak) <= 0 :
            # unbounded solution
            print('Unbounded solution!')
            iter=-1
        else:
            # calc leaving var, utgvar
            # --------
            alkHatt= np.divide(b,np.transpose(N)[inkvar]) #inkommande var kol genom b (bhat), nu bestäms utgående av det minsta
            print("alkHatt: ", alkHatt)

            utgvarPlats = int(np.where(alkHatt == np.amin(alkHatt))[0][0])
            utgvar = bix[utgvarPlats]
    


            print(' Iter: '+repr(iter)+' z: '+repr(z)+' rc: '+repr(rc_min)+' ink: '+repr(inkvar+1)+' utg: '+repr(utgvar+1))

            # make new partition
            # -------
            temp = bix[utgvarPlats]
            bix[utgvarPlats] = nix[inkvar]
            nix[inkvar] = temp

            B = A[:,bix]
            N = A[:,nix]
            cB = c[bix]
            cN = c[nix]
            

   



elapsed = time.time() - t1
print('Elapsed time: '+repr(elapsed))

Rows: 3 cols: 5
bhatt origin:  [ 1.    0.75 10.  ]
bix origin:  [2 3 4]
ak:  [1. 1. 8.]
alkHatt:  [1.   0.75 1.25]
 Iter: 1 z: 0.0 rc: -2.0 ink: 1 utg: 4
bhatt origin:  [0.25 0.75 4.  ]
bix origin:  [2 0 4]
ak:  [ 1.  0. 20.]
alkHatt:  [1.  inf 0.5]
 Iter: 2 z: -1.5 rc: -1.0 ink: 2 utg: 5
bhatt origin:  [0.05 0.75 0.2 ]
bix origin:  [2 0 1]
Ready
bhatt:  [0.05 0.75 0.2 ]
t:  [0.75 0.2  0.05 0.   0.  ]
funktionsvärde:  -1.7000000000000002
xdiff: 3.1031676915590914e-17
zdiff: -2.220446049250313e-16
Elapsed time: 0.0040013790130615234


C:\Users\marcu\AppData\Local\Temp\ipykernel_31416\3871641781.py:86: RuntimeWarning: divide by zero encountered in divide
  alkHatt= np.divide(b,np.transpose(N)[inkvar]) #inkommande var kol genom b (bhat), nu bestäms utgående av det minsta
